# Descrição da Classe AutomatosFinitos

A classe `AutomatosFinitos` implementa um automato finito determinístico (AFD) com base num padrão de procura, permitindo encontrar todas as ocorrências desse padrão dentro de um texto, além de visualizar a tabela de transições e o caminho de estados percorridos.

---

## Projeto de Alto Nível (AutomatosFinitos)

### 1. **Entrada e Saída**:
- **Entrada:**
    - Um alfabeto (set de símbolos).
    - Um padrão (str) a ser reconhecido no texto.
- **Saída:**
    - Lista de estados percorridos (process_input).
    - Lista de posições de ocorrências (find_occurrences).
    - Impressão da tabela de transição (visualize).

### 2. **Componentes Principais**:
- ` __init__`:
    - Inicializa o automato com o alfabeto e o padrão.
    - Cria a tabela de transições.
- `_build_transition_table`:
    - Constrói a tabela de transições entre estados com base no padrão.
- ` _calculate_state`:`
    - Calcula o próximo estado ideal dado um símbolo e o estado atual, considerando sobreposição máxima com o início do padrão.
- `process_input`:
    - Percorre a sequência de entrada, seguindo as transições do automato, e retorna os estados visitados.
- `find_occurrences`:
    - Encontra todas as posições no texto onde o padrão ocorre completamente.
- `visualize`:
    - Imprime a estrutura do automato (estados e transições).
- `get_next_state`:
    - Retorna o próximo estado, validando o símbolo contra o alfabeto.
---
## Projeto de Baixo Nível 

### 1. **Construção da Tabela de Transição**:
- **Descrição:**
    - Para cada estado e cada símbolo do alfabeto, define o próximo estado com base na maior sobreposição entre o sufixo atual e o prefixo do padrão.
- **Algoritmo:**
    - Itera sobre todos os pares (estado, símbolo).
    - Usa `_calculate_state` para determinar o estado de destino.


### 2. **Cálculo do Estado**:
- **Descrição:**
    - Dado um estado atual e símbolo, calcula o próximo estado simulando a adição do símbolo ao sufixo do padrão visto até o momento.
- **Algoritmo:**
    - Junta o prefixo do padrão ao estado atual com o novo símbolo.
    - Verifica a maior sobreposição com o início do padrão.


### 3. **Processamento da Entrada**:
- **Descrição:**
    - Simula a execução do automato sobre uma sequência.
- **Algoritmo:**
    - Começa do estado 0.
    - Para cada caractere da sequência, aplica a transição e armazena o estado visitado.


### 4. **Busca de Ocorrências**:
- **Descrição:**
    - Identifica todos os pontos do texto onde o padrão foi completamente reconhecido.
- **Algoritmo:**
    - Executa `process_input`.
    - Se o estado final de algum ponto for igual ao número de caracteres do padrão, significa que houve uma ocorrência.


### 5. **Visualização**:
- **Descrição:**
    - Imprime todos os estados e suas transições de forma organizada.
- **Algoritmo:**
    - Itera sobre todos os pares (estado, símbolo).
    - Usa _calculate_state para determinar o estado de destino.

## 6. **Transição segura**
- **Descrição:**
    - Obter com segurança o próximo estado com base no símbolo lido.
- **Algoritmo:**
    - Verifica se o símbolo está no alfabeto (evita erros).
    - Retorna o próximo estado com base na tabela de transição ou 0.

## 7. **Cálculo de Sobreposição entre Padrões** 
- **Descrição:**
    - Calcular o maior "overlap" entre o final de uma string e o início do padrão.
- **Algoritmo:**
    - Compara sufixos do `suffix_candidate` com prefixos do `prefix_source`.
    - Quanto maior o valor retornado, mais "longo" é o trecho que pode continuar o reconhecimento do padrão.

---


## Implementação




In [6]:
class AutomatosFinitos :

    def __init__(self, alphabet: set, pattern: str):
        if not pattern:
            raise ValueError ('Pattern cannot be empty')
        if not alphabet:
            raise ValueError ('Alphabet cannot be empty')
        
        self.num_states = len(pattern) + 1
        self.alphabet = alphabet
        self.pattern = pattern
        self.transition_table = {}
        self._build_transition_table()

    def _build_transition_table(self):

        """Constructs the transition table using optimized overlap calculation."""

        for state in range(self.num_states):
            for symbol in self.alphabet:
                next_state = self._calculate_state(state, symbol)
                self.transition_table[(state, symbol)] = next_state

    def _calculate_state(self, current_state: int, symbol: str):
        """Calculates the next state with memoization for repeated patterns."""

        if current_state == 0 and symbol == self.pattern[0]:
            return 1
        
        candidate = self.pattern[:current_state] +  symbol
        max_overlap = min(current_state + 1, len(self.pattern))

        for length in range(max_overlap, 0, -1):
            if candidate.endswith(self.pattern[:length]):
                return length
        return 0
    
    def process_input(self, sequence: str) :
        """Processes an input sequence and returns states visited."""
        
        current_state = 0
        states_log = [current_state]

        for char in sequence:
            current_state = self.transition_table.get(
                (current_state, char), 0
            )
            states_log.append(current_state)
        return states_log
    
    def find_occurrences (self, text: str):
        """Finds all pattern occurrences with their starting positions."""

        states = self.process_input(text)
        pattern_length = len(self.pattern)
        matches = [
            index - pattern_length 
            for index, state in enumerate(states)
            if state == len (self.pattern)
        ]
        return len (matches), matches
    
    def visualize (self):
        print(f"States: {self.num_states}")
        print(f"Search Pattern: {self.pattern}")
        print("Transition Map:")
        for state in sorted(set(s for s, _ in self.transition_table)):
            symbols = [k[1] for k in self.transition_table if k[0] == state]
            for symbol in sorted(symbols):
                dest = self.transition_table[(state, symbol)]
                print(f"  ({state}, {symbol}) → {dest}")

    def get_next_state(self, current_state: int, symbol: str):
        """Safe state transition with input validation."""
        if symbol not in self.alphabet:
            raise ValueError(f"Symbol '{symbol}' not in alphabet")
        return self.transition_table.get((current_state, symbol), 0)
    
def pattern_overlap(suffix_candidate: str, prefix_source: str):
    """Calculates maximum overlap between suffix and prefix."""
    max_length = min(len(suffix_candidate), len(prefix_source))
    for length in range(max_length, 0, -1):
        if suffix_candidate[-length:] == prefix_source[:length]:
            return length
    return 0

In [7]:
alphabet = {'a', 'b', 'c'}
pattern = "abc"

# Create the finite automaton
automaton = AutomatosFinitos(alphabet, pattern)

# Visualize the automaton's transition table
print("Visualizing the automaton:")
automaton.visualize()

# Define a text to search for the pattern
text = "abcabcababc"

# Find occurrences of the pattern in the text
count, positions = automaton.find_occurrences(text)

# Output results
print("\nResults:")
print(f"Text: {text}")
print(f"Pattern: {pattern}")
print(f"Number of occurrences: {count}")
print(f"Starting positions of occurrences: {positions}")

# Process the input sequence and show states visited
states_visited = automaton.process_input(text)
print(f"\nStates visited during processing: {states_visited}")

Visualizing the automaton:
States: 4
Search Pattern: abc
Transition Map:
  (0, a) → 1
  (0, b) → 0
  (0, c) → 0
  (1, a) → 1
  (1, b) → 2
  (1, c) → 0
  (2, a) → 1
  (2, b) → 0
  (2, c) → 3
  (3, a) → 1
  (3, b) → 0
  (3, c) → 0

Results:
Text: abcabcababc
Pattern: abc
Number of occurrences: 3
Starting positions of occurrences: [0, 3, 8]

States visited during processing: [0, 1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 3]


# Projeto de Alto Nível – Burrows-Wheeler Transform (BWT)

### **Projeto de Alto Nível**

1. **Entrada e Saída**:
   - **Entrada**: Uma sequência de caracteres (como DNA ou texto), sem o símbolo de fim.
   - **Saída**: Uma lista com:
     - A sequência original;
     - A sequência transformada pela BWT;
     - A posição da sequência original na matriz de rotações ordenadas.

2. **Componentes Principais**:
   - **Adição do Terminador**: Acrescenta um símbolo especial (`$`) ao final da sequência para garantir unicidade.
   - **Geração das Rotações**: Cria todas as rotações circulares possíveis da sequência com o terminador.
   - **Ordenação Lexicográfica**: Ordena as rotações em ordem alfabética.
   - **Construção da BWT**: Extrai o último caractere de cada rotação ordenada para formar a sequência BWT.
   - **Identificação da Posição Original**: Determina a posição da sequência original dentro da lista ordenada.

3. **Fluxo de Dados**:
   Entrada da sequência → Adição de `$` → Geração das rotações → Ordenação → Extração da última coluna → Localização do índice original → Saída final em lista `[original, bwt, índice]`.

---

### **Projeto de Baixo Nível**

1. **Adição de Terminador (`$`)**:
   - **Descrição**: Adiciona um caractere especial ao final da sequência para garantir que nenhuma outra rotação seja idêntica à original. Isso também facilita a reconstrução posterior da sequência.

2. **Geração de Rotações Cíclicas**:
   - **Descrição**: Cria todas as possíveis rotações da sequência, deslocando os caracteres circularmente. Cada rotação representa uma possível reorganização da sequência.

3. **Ordenação das Rotações**:
   - **Descrição**: Ordena as rotações em ordem lexicográfica (alfabética), o que é essencial para a formação da BWT e permite compactação e busca mais eficiente.

4. **Construção da Sequência BWT**:
   - **Descrição**: Forma a sequência BWT pegando o último caractere de cada rotação ordenada. Essa nova sequência tende a agrupar caracteres similares, favorecendo compressão.

5. **Identificação da Posição Original**:
   - **Descrição**: Determina a posição da sequência original (com o `$`) na lista ordenada de rotações, essencial para reverter a BWT corretamente.

6. **Reversão da BWT (Reconstrução da Sequência)**:
   - **Descrição**: Reconstrói a sequência original com base na BWT e na posição original. Para isso, constrói iterativamente a matriz de rotações até identificar a linha correspondente à sequência original e remove o terminador ao final.

Code for PWM

In [8]:
from typing import List, Union

def bwt_transform(seq: str) -> List[Union[str, int]]:
    """
    Executa a transformação de Burrows-Wheeler (BWT) numa sequência.

    Parâmetros:
        seq (str): A sequência de entrada (sem o terminador).

    Retorna:
        List[Union[str, int]]: Uma lista com:
            - a sequência original;
            - a sequência transformada BWT;
            - o índice da sequência original na lista de rotações ordenadas.
    """
    original = seq  # Guarda a original
    if '$' not in seq:
        seq += '$'

    rotacoes = [seq[i:] + seq[:i] for i in range(len(seq))]
    rotacoes.sort()
    bwt = ''.join(rot[-1] for rot in rotacoes)

    return [original, bwt] 


def bwt_reverse(bwt: str) -> str:
    """
    Reverte a transformação BWT e recupera a sequência original.

    Parâmetros:
        bwt (str): A sequência transformada pela BWT.
        original_index (int): Índice da sequência original na matriz ordenada.

    Retorna:
        str: A sequência original (sem o terminador).
    """
    n = len(bwt)
    table = [''] * n
    start_index = bwt.index("$")

    for _ in range(n):
        # Adiciona bwt como prefixo a cada linha da tabela
        table = sorted([bwt[i] + table[i] for i in range(n)])

    # A linha original é aquela no índice original_index
    original = table[start_index]

    # Remover o símbolo de fim '$' antes de devolver
    return original.rstrip('$')


resultado = bwt_transform("banana")
print(resultado)
original_recuperado = bwt_reverse(resultado[1])
print("Sequência recuperada:", original_recuperado)




['banana', 'annb$aa']
Sequência recuperada: banana


In [14]:
from typing import List, Union

class BWT:
    def __init__(self, seq: str, buildsufarray: bool = False):
        """
        Inicializa a classe BWT e constrói a transformada BWT e o Suffix Array (opcional).
        """
        self.seq = seq + "$" if "$" not in seq else seq
        self.bwt = self.build_bwt(self.seq)
        self.sa = self.build_suffix_array(self.seq) if buildsufarray else []

    def build_bwt(self, text: str) -> str:
        """
        Constrói a transformada de Burrows-Wheeler (BWT) para uma sequência.
        """
        rotations = [text[i:] + text[:i] for i in range(len(text))]
        rotations.sort()
        return ''.join(rot[-1] for rot in rotations)

    def build_suffix_array(self, text: str) -> List[int]:
        """
        Constrói o Suffix Array para uma sequência.
        """
        suffixes = [(text[i:], i) for i in range(len(text))]
        suffixes.sort()  # Ordena os sufixos lexicograficamente
        return [suffix[1] for suffix in suffixes]

    def last_to_first(self) -> List[int]:
        """
        Cria o mapeamento da última coluna para a primeira coluna na matriz BWT.
        """
        first_col = sorted(self.bwt)
        last_to_first_mapping = []
        
        count = {}
        for i, char in enumerate(self.bwt):
            count[char] = count.get(char, 0) + 1
            occurrence_index = count[char]
            pos_in_first_col = find_ith_occurrence(first_col, char, occurrence_index)
            last_to_first_mapping.append(pos_in_first_col)
        
        return last_to_first_mapping

    def bw_matching(self, pattern: str) -> List[int]:
        """
        Procura todas as ocorrências de um padrão na sequência original usando BWT.
        
        Retorna os índices das linhas da matriz M onde o padrão ocorre.
        """
        lf_mapping = self.last_to_first()
        
        top = 0
        bottom = len(self.bwt) - 1
        
        while top <= bottom and pattern:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            
            sub_bwt = self.bwt[top:bottom + 1]
            
            if symbol in sub_bwt:
                top_index = sub_bwt.index(symbol) + top
                bottom_index = bottom - sub_bwt[::-1].index(symbol)
                
                top = lf_mapping[top_index]
                bottom = lf_mapping[bottom_index]
            else:
                return []  # Padrão não encontrado
        
        return list(range(top, bottom + 1))

    def bw_matching_pos(self, pattern: str) -> List[int]:
        """
        Procura as posições iniciais do padrão na sequência original usando o Suffix Array.
        
        Retorna uma lista com as posições iniciais ordenadas.
        """
        matches = self.bw_matching(pattern)
        
        if not matches:
            return []  # Nenhuma ocorrência encontrada
        
        positions = [self.sa[m] for m in matches]
        
        positions.sort()
        
        return positions
    
    #  Função auxiliar para encontrar a n-ésima ocorrência de um símbolo em uma lista
    def find_ith_occurrence(lst: List[str], elem: str, index: int) -> int:
        count = 0
        for i, c in enumerate(lst):
            if c == elem:
                count += 1
                if count == index:
                    return i
        return -1


# Teste completo
if __name__ == "__main__":
    seq = "BANANA"
    bwt_instance = BWT(seq, buildsufarray=True)

    print(f'[ {seq} , {bwt_instance.bwt} ]')
    print("Suffix Array:", bwt_instance.sa)

    pattern = "ANA"
    positions = bwt_instance.bw_matching_pos(pattern)
    count = len(positions)

    
    print(f"'{pattern}', {count}", positions)


[ BANANA , ANNB$AA ]
Suffix Array: [6, 5, 3, 1, 0, 4, 2]
'ANA', 2 [1, 3]


# Projeto de Alto Nível – Tries and Suffix Trees

In [10]:
class Trie:
    def __init__(self) -> None:
        """
        Initializes an empty Trie with a root node (node 0).
        """
        self.nodes = {0: {}}  # root node
        self.num = 0          # node counter

    def print_trie(self) -> None:
        """
        Prints the structure of the Trie for debugging.
        """
        for k in self.nodes.keys():
            print(k, "->", self.nodes[k])

    def add_node(self, origin: int, symbol: str) -> None:
        """
        Adds a new node to the Trie.

        :param origin: ID of the node where the edge starts
        :param symbol: character for the edge label
        """
        self.num += 1
        self.nodes[origin][symbol] = self.num
        self.nodes[self.num] = {}

    def add_pattern(self, p: str) -> None:
        """
        Inserts a single pattern into the Trie.

        :param p: pattern to insert
        """
        pos = 0
        node = 0
        while pos < len(p):
            if p[pos] not in self.nodes[node]:
                self.add_node(node, p[pos])
            node = self.nodes[node][p[pos]]
            pos += 1

    def trie_from_patterns(self, pats: list[str]) -> None:
        """
        Inserts a list of patterns into the Trie.

        :param pats: list of patterns
        """
        for p in pats:
            self.add_pattern(p)

    def prefix_trie_match(self, text: str) -> str | None:
        """
        Searches for the longest prefix in the Trie that matches the beginning of text.

        :param text: input string to match
        :return: matched prefix string or None
        """
        pos = 0
        match = ""
        node = 0
        while pos < len(text):
            char = text[pos]
            if char in self.nodes[node]:
                node = self.nodes[node][char]
                match += char
                if self.nodes[node] == {}:  # terminal node
                    return match
                pos += 1
            else:
                return None
        return None

    def trie_matches(self, text: str) -> list[tuple[int, str]]:
        """
        Searches for all patterns in the Trie that match substrings in text.

        :param text: the input string
        :return: list of tuples (start_position, matched_string)
        """
        res = []
        for i in range(len(text)):
            match = self.prefix_trie_match(text[i:])
            if match is not None:
                res.append((i, match))
        return res


In [11]:
def test():
    patterns = ["GAT", "CCT", "GAG"]
    t = Trie()
    t.trie_from_patterns(patterns)
    t.print_trie()

def test2():
    patterns = ["AGAGAT", "AGC", "AGTCC", "CAGAT", "CCTA", "GAGAT", "GAT", "TC"]
    t = Trie()
    t.trie_from_patterns(patterns)
    print("Match único:", t.prefix_trie_match("GAGATCCTA"))
    print("Todos os matches:", t.trie_matches("GAGATCCTA"))

if __name__ == "__main__":
    print("Test 1:")
    test()
    print("\nTest 2:")
    test2()
    print("\nDONE")


Test 1:
0 -> {'G': 1, 'C': 4}
1 -> {'A': 2}
2 -> {'T': 3, 'G': 7}
3 -> {}
4 -> {'C': 5}
5 -> {'T': 6}
6 -> {}
7 -> {}

Test 2:
Match único: GAGAT
Todos os matches: [(0, 'GAGAT'), (2, 'GAT'), (4, 'TC'), (5, 'CCTA')]

DONE


In [12]:
class SuffixTree:
    def __init__(self):
        """
        Initializes the suffix tree with a root node.
        Each node is represented as a tuple: (leaf_index, children_dict).
        """
        self.nodes = {0: (-1, {})}  # node_id: (leaf_index or -1, {symbol: child_node_id})
        self.node_count = 0
        self.text = ""

    def _add_node(self, parent, symbol, leaf_index=-1):
        """
        Adds a new node as a child of the given parent node.

        :param parent: index of the parent node
        :param symbol: character representing the edge to the new node
        :param leaf_index: suffix index if it's a leaf node, otherwise -1
        """
        self.node_count += 1
        self.nodes[parent][1][symbol] = self.node_count
        self.nodes[self.node_count] = (leaf_index, {})

    def _add_suffix(self, suffix, index):
        """
        Adds a suffix to the tree.

        :param suffix: suffix string to be inserted
        :param index: starting index of the suffix in the original text
        """
        node = 0
        for i, char in enumerate(suffix):
            if char not in self.nodes[node][1]:
                is_leaf = (i == len(suffix) - 1)
                self._add_node(node, char, index if is_leaf else -1)
            node = self.nodes[node][1][char]

    def build(self, text):
        """
        Builds the suffix tree from the given input string.

        :param text: input string to build the suffix tree from
        """
        self.text = text + "$"
        for i in range(len(self.text)):
            self._add_suffix(self.text[i:], i)

    def _get_leaves_below(self, node_id):
        """
        Recursively collects all leaf indices under the given node.

        :param node_id: ID of the starting node
        :return: list of leaf indices
        """
        leaf_index, children = self.nodes[node_id]
        if leaf_index >= 0:
            return [leaf_index]

        results = []
        for child in children.values():
            results.extend(self._get_leaves_below(child))
        return results

    def find_pattern(self, pattern):
        """
        Searches for a pattern in the suffix tree.

        :param pattern: string pattern to search for
        :return: list of starting positions in the text where the pattern occurs, or None
        """
        node = 0
        for char in pattern:
            if char in self.nodes[node][1]:
                node = self.nodes[node][1][char]
            else:
                return None
        return self._get_leaves_below(node)

    def print_tree(self):
        """
        Prints the structure of the suffix tree.
        """
        for node_id, (leaf_index, children) in self.nodes.items():
            if leaf_index >= 0:
                print(f"{node_id} : leaf index {leaf_index}")
            else:
                print(f"{node_id} -> children: {children}")


In [13]:
if __name__ == "__main__":
    st = SuffixTree()
    st.build("banana")
    st.print_tree()

    print("Matches for 'ana':", st.find_pattern("ana"))
    print("Matches for 'na':", st.find_pattern("na"))
    print("Matches for 'a':", st.find_pattern("a"))
    print("Matches for 'x':", st.find_pattern("x"))


0 -> children: {'b': 1, 'a': 8, 'n': 14, '$': 22}
1 -> children: {'a': 2}
2 -> children: {'n': 3}
3 -> children: {'a': 4}
4 -> children: {'n': 5}
5 -> children: {'a': 6}
6 -> children: {'$': 7}
7 : leaf index 0
8 -> children: {'n': 9, '$': 21}
9 -> children: {'a': 10}
10 -> children: {'n': 11, '$': 19}
11 -> children: {'a': 12}
12 -> children: {'$': 13}
13 : leaf index 1
14 -> children: {'a': 15}
15 -> children: {'n': 16, '$': 20}
16 -> children: {'a': 17}
17 -> children: {'$': 18}
18 : leaf index 2
19 : leaf index 3
20 : leaf index 4
21 : leaf index 5
22 : leaf index 6
Matches for 'ana': [1, 3]
Matches for 'na': [2, 4]
Matches for 'a': [1, 3, 5]
Matches for 'x': None
